## Import Libraries

In [ ]:
import scipy.io
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Concatenate, Dropout, Flatten
from keras import regularizers 
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras import backend as K

## Mount Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)
DataDir = '/content/drive/My Drive/DGF/'
InjDir = DataDir + '/injections/'
CgmDir = DataDir + '/cgm/'
ChoDir = DataDir + '/cho/' 
imagespath = '/content/drive/My Drive/DGF/Results/'

## Custom Functions

In [ ]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('inj(t-%d)' %i), ('cho(t-%d)' %i), ('cgm(t-%d)' %i)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('inj(t)'), ('cho(t)'), ('cgm(t)')]
        else:
            names += [('inj(t+%d)' %i), ('cho(t+%d)' %i), ('cgm(t+%d)' %i)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

## Import dataset

In [ ]:
ins = []
cgm = []
cho = []

i = 1
g = 1
c = 1

for signal in os.listdir(InjDir):
    ins.append(scipy.io.loadmat(InjDir+str(i)+".mat"))
    i += 1
for signal in os.listdir(CgmDir): 
    cgm.append(scipy.io.loadmat(CgmDir+str(g)+".mat"))
    g += 1
for signal in os.listdir(ChoDir):    
    cho.append(scipy.io.loadmat(ChoDir+str(c)+".mat"))
    c += 1

## Extract NpArrays from dictionaries

In [ ]:
Np = 2
injections = []
cgms = []
carbos = []

for i in range (0, Np):
    injection = ins[i]['injections'].flatten()
    injections.append(injection)
    glucose = cgm[i]['cgm'].flatten()
    cgms.append(glucose)
    carbo= cho[i]['cho'].flatten()
    carbos.append(carbo)

#Transform lists into 1D-NpArrays
Injections = np.array(injections[0][1:])
Cgms = np.array(cgms[0][1:])
Carbos = np.array(carbos[0][1:])

## Transform sequences to supervised learning samples

In [ ]:
raw = DataFrame()
raw['ob1'] = Injections
raw['ob2'] = Carbos
raw['ob3'] = Cgms
values = raw.values
values = values.astype('float32')
#Specify the number of features and the 'look back' and 'prediction horizon' parameters (Only ph2 steps in the future are 'known', out of ph1)
lb = 5
ph1 = 5
ph2 = 5 
nbr_features = 3
#Transform data into series for supervised learning
data = series_to_supervised(values, lb, ph1)
print(data)

## Split samples in train, validation and test values

In [ ]:
# split into train and test sets
values = data.values
train_data_len = 24*60*14
valid_data_len = 24*60*7
train = values[:train_data_len, :]
valid = values[train_data_len:(train_data_len+valid_data_len), :]
test = values[(train_data_len+valid_data_len):, :]

# split into input and outputs
nbr_obs = nbr_features*lb
x_train1, x_train2, y_train = train[:, :nbr_obs], np.delete(np.delete(np.delete(train, np.s_[(nbr_obs+2)::3], axis=1), np.s_[:nbr_obs], axis=1), np.s_[2*ph2:], axis=1), train[:, (nbr_obs+2)::3]
x_valid1, x_valid2, y_valid = valid[:, :nbr_obs], np.delete(np.delete(np.delete(valid, np.s_[(nbr_obs+2)::3], axis=1), np.s_[:nbr_obs], axis=1), np.s_[2*ph2:], axis=1), valid[:, (nbr_obs+2)::3]
x_test1, x_test2, y_test = test[:, :nbr_obs], np.delete(np.delete(np.delete(test, np.s_[(nbr_obs+2)::3], axis=1), np.s_[:nbr_obs], axis=1), np.s_[2*ph2:], axis=1), test[:, (nbr_obs+2)::3]

# reshape input to be 3D of the format [samples, timesteps, features]
x_train1 = x_train1.reshape((x_train1.shape[0], lb, nbr_features))
x_train2 = x_train2.reshape((x_train2.shape[0], ph2, nbr_features-1))
x_valid1 = x_valid1.reshape((x_valid1.shape[0], lb, nbr_features))
x_valid2 = x_valid2.reshape((x_valid2.shape[0], ph2, nbr_features-1))
x_test1 = x_test1.reshape((x_test1.shape[0], lb, nbr_features))
x_test2 = x_test2.reshape((x_test2.shape[0], ph2, nbr_features-1))

## Build the Deep GLucose Forcasting architecture

In [ ]:
#Define the first RNN
first_model = Sequential()
first_model.add(LSTM(64, input_shape=(lb, nbr_features), return_sequences=False))

#Define the second RNN
second_model = Sequential()
second_model.add(LSTM(64, input_shape=(ph2, nbr_features-1), return_sequences=False))

#Concatenate the two models' outputs and pass it to the overall output layer
MergedOutput = Concatenate()([first_model.output, second_model.output])
MergedOutput = Dense(ph1, activation='relu')(MergedOutput)

#Generate the overall model
final_model = Model([first_model.input, second_model.input], MergedOutput)

#Model summary
final_model.summary()

#Compile the model
opt = optimizers.Adam(learning_rate=0.001)
final_model.compile(optimizer=opt, loss='mean_squared_error')

## Train the network

In [ ]:
#Train the model on the adult patient
stopping = EarlyStopping(monitor='val_loss', patience=4)
history = final_model.fit([x_train1, x_train2], y_train, batch_size=40, epochs=300, validation_data=([x_valid1, x_valid2], y_valid), callbacks=[stopping], verbose=2, shuffle=True)

## Save the Model

In [ ]:
models_dir = DataDir + '/Models/'

def savemodel(model, name):
    filename = os.path.join(models_dir, '%s.h5' %name)
    final_model.save(filename)
    print("\nModel saved successfully on file %s\n" %filename)

# Save the model
savemodel(final_model, 'Full horizon 5')

## Plot training history

In [ ]:
#Plot training history
plt.figure(figsize=(9,6))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='valid')
plt.legend()
plt.xlabel('Epochs', fontdict=dict(size='12'))
plt.ylabel('Loss (MSE)', fontdict=dict(size='12'))
#plt.savefig('ph30.png')
plt.show()

## Make predictions

In [ ]:
#Calculate the model predictions
predictions = final_model.predict([x_test1, x_test2])

#Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print('Test RMSE: %.3f' % rmse)

## Plot the predicted Glucose values vs the actual ones

In [ ]:
#Create the true and predicted glucose histories
Predictions=[]
for i in range (0, len(predictions)):
    Predictions.append(predictions[i][4])

Validation=[]
for i in range (0, len(y_test)):
    Validation.append(y_test[i][4])

#Convertion to Numpy arrays
Predictions=np.array(Predictions)
Validation=np.array(Validation)

#Plot the results
plt.figure(figsize=(14,7))
plt.plot(Predictions[0:1440], 'r')
plt.plot(Validation[0:1440], 'b')
plt.xlabel('Time [min]', fontdict=dict(size='12'))
plt.ylabel('Glucose Concentration [mg/dl]', fontdict=dict(size='12'))
plt.legend(['Prediction', 'Actual'], loc='upper right')
#plt.savefig('5_15_5min.png')
plt.show()